## OCNS 2019, Simulating dendrites at different levels of abstraction.
## How inputs propagate to the soma and how inputs interact.
### In this exercise we will see how inputs attenuate towards the soma. Also we will see how inputs integrate in a dendritic branch. 

In [ ]:
# -*- coding: utf-8 -*-
from neuron import h, gui
from matplotlib import pyplot as plt
% matplotlib inline
fs = 14  # font size for plots

#Simulation parameters	
h.tstop=100 		        #ms
h.dt=0.1			        #ms
h.steps_per_ms=10		#For plotting-should be compartible with dt
h.v_init=-65

### Create a soma and two dendrites, define their anatomical and biophysical properties and connect them.

In [ ]:
#------------------------------------------------------
soma=h.Section(name='soma')
dend0, dend1 = [h.Section(name=n) for n in ['dend0', 'dend1']]

# Define properties of soma
soma.diam = 20	#um
soma.L = 200    	#um
soma.Ra = 100         # Axial resistance in Ohm * cm
soma.cm = 1		      #specific membrane capacitance, capacitance per unit of membrane, uF/cm^2   
soma.nseg=1	

# Insert the hh channels (including leak)
soma.insert('hh')
for seg in soma: 
    seg.hh.gnabar = 0.12  # Sodium conductance in S/cm2
    seg.hh.gkbar = 0.036  # Potassium conductance in S/cm2
    seg.hh.gl = 0.0003    # Leak conductance in S/cm2
    seg.hh.el = -65       # Reversal potential in mV

# Define properties of dend0
dend0.diam = 5
dend0.L = 500
dend0.nseg=11
dend0.cm = 1
dend0.Ra = 100

#Insert passive (leak) channels
dend0.insert('pas')      
for seg in dend0:
    seg.pas.e= -65
    seg.pas.g= 0.0003
	
# Define properties of dend1
dend1.diam = 3
dend1.L = 200
dend1.nseg=13
dend1.cm = 1
dend1.Ra = 100

#Insert passive (leak) channels
dend1.insert('pas')     
for seg in dend0:
    seg.pas.e= -65
    seg.pas.g= 0.0003

#Connect the 0 point of dendrite 0 to the zero point of the soma
dend0.connect(soma(0),0)        
dend1.connect(dend0(1),0)  

### Synaptic Stimulation

In [ ]:
#Place the synapse to the middle (0.5) of the soma, ...
Syn=h.ExpSyn(soma(0.5))
Syn.e=0                     #Reverse potential of the synapse (mV)

#...create an artificial spike (an "event" to be delivered to the synapse)...
ns=h.NetStim(0.5)
ns.start = 50      
ns.number = 1  

#... and connect the event to the synapse.
nc = h.NetCon(ns,Syn, -10, 2, 0.008*5)

### Example simulation
### Calculate and print in terminal the depolarization at the soma and at the middle part of the distal dendrite (dendrite[1])

In [ ]:
vsoma_vec = h.Vector()        # Membrane potential vector
vdend1_vec = h.Vector()        # Membrane potential vector
t_vec = h.Vector()        # Time stamp vector
vsoma_vec.record(soma(0.5)._ref_v)
vdend1_vec.record(dend1(0.5)._ref_v)
t_vec.record(h._ref_t)

# Run the simulation
h.run()

vsoma_vec.remove(0,200)
vdend1_vec.remove(0,200)
t_vec.remove(0,200)

plt.figure(figsize=(8,4)) # Default figsize is (8,6)
soma_plot = plt.plot(t_vec, vsoma_vec, color='black')
dend_plot = plt.plot(t_vec, vdend1_vec, color='red')

plt.legend(soma_plot + dend_plot, ['soma', 'dend'])
plt.xlabel('time (ms)')
plt.ylabel('mV')

print ('Somatic depolarization is ',  round(vsoma_vec.max()-vsoma_vec.min(),2), ' mV')
print ('Distal Dendritic depolarization is', round(vdend1_vec.max()-vdend1_vec.min(),2), 'mV')

### Task 1: How can you easily make this synapse inhibitory?

### Task 2: Move the synapse to the middle of dendrite[1] section.
### What do you expect for the amplitude and the duration of the EPSP seen at the dendrite and at the soma?

!['Every input for itself (the passive dendritic tree)'](Picture1.png)
#### Spruston, Nature Reviews, 2008

### Task 3: Insert 1 to 5 synapses like this one in the same location. In this case, this is equivalent to increasing the weight 1 to 5 times.
### What is the expected (arithmetic sum) versus actual (simulated output)?
### Is their relationship linear? 
![''](Picture2.jpg)